# <strong>Bitcoin Recurrent Neural Network<strong>
### Justin Marlor & Habit Blunk
##### *Colorado State University*

In [ ]:
import pandas as pd
import subprocess
import matplotlib.pyplot as plt

result = subprocess.run(['bash', './add-run-kaggle-bitcoin'], capture_output=True,text=True)

print(result.stdout)
print(result.stderr)

if result.returncode == 0:
  df = pd.read_csv("kaggle-bitcoin/upload/btcusd_1-min_data.csv", dtype={"Volume": float}, low_memory=False)
  display(df)

In [ ]:
df['datetime'] = pd.to_datetime(df['Timestamp'].astype('Int64'), unit='s', errors='coerce')
df['Year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
display(df)

plt.plot(df['datetime'], df['Open'], label='open', color='blue')
plt.plot(df['datetime'], df['Close'], label='close', color='green')
plt.plot(df['datetime'], df['High'], label='high', color='red')
plt.plot(df['datetime'], df['Low'], label='low', color='orange')

plt.xlabel('datetime')
plt.ylabel('price')
plt.title('ohlc time series')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import torch
import torch.nn as nn
from torch.nn import GRU
from torch.nn import RNN

input_size = df.shape[1]

rnn = nn.RNN(input_size=input_size, hidden_size=50, num_layers=2, batch_first=True)